In [1]:
import requests, os, re
from bs4 import BeautifulSoup
import pandas as pd

In [13]:
# Creat list of URLs of the APIs to call in order to get pas season stats
years = list(range(1990,2024))
api_urls = [f"https://stats.nba.com/stats/leagueLeaders?LeagueID=00&PerMode=PerGame&Scope=S&Season={year}-{str(year+1)[-2:]}&SeasonType=Regular%20Season&StatCategory=PTS" for year in years]

'https://www.nba.com/stats/leaders?Season=1990-91&SeasonType=Regular%20Season'